In [7]:
import torch
from policyengine_uk import Microsimulation
import pandas as pd
import numpy as np
from tqdm import tqdm

# Fill in missing constituencies with average column values
import pandas as pd
import numpy as np

from policyengine_uk_data.utils.loss import (
    create_target_matrix as create_national_target_matrix,
)

import h5py

ages = pd.read_csv("targets/age.csv")
incomes = pd.read_csv("targets/total_income.csv")
earnings = pd.read_csv("targets/employment_income.csv")

ENGLAND_CONSTITUENCY = "E14"
NI_CONSTITUENCY = "N06"
SCOTLAND_CONSTITUENCY = "S14"
WALES_CONSTITUENCY = "W07"

incomes = incomes[
    np.any(
        [
            incomes["code"].str.contains(country_code)
            for country_code in [
                ENGLAND_CONSTITUENCY,
                NI_CONSTITUENCY,
                SCOTLAND_CONSTITUENCY,
                WALES_CONSTITUENCY,
            ]
        ],
        axis=0,
    )
]

full_constituencies = incomes.code
missing_constituencies = pd.Series(list(set(incomes.code) - set(ages.code)))
missing_constituencies = pd.DataFrame(
    {
        "code": missing_constituencies.values,
        "name": incomes.set_index("code")
        .loc[missing_constituencies]
        .name.values,
    }
)
for col in ages.columns[2:]:
    missing_constituencies[col] = ages[col].mean()

ages = pd.concat([ages, missing_constituencies])

sim = Microsimulation()

employment_income_bounds = earnings.employment_income_lower_bound.unique()

print(employment_income_bounds)


def create_target_matrix():
    matrix = pd.DataFrame()
    y = pd.DataFrame()

    total_income = sim.calculate("total_income", period=2025).values
    matrix["hmrc/total_income_amount"] = sim.map_result(
        total_income, "person", "household"
    )
    y["hmrc/total_income_amount"] = incomes["total_income_amount"]

    matrix["hmrc/total_income_count"] = sim.map_result(
        total_income != 0, "person", "household"
    )
    y["hmrc/total_income_count"] = incomes["total_income_count"]

    employment_income = sim.calculate("employment_income", period=2025).values
    
    for lower_bound, upper_bound in zip(employment_income_bounds[:-1], employment_income_bounds[1:]):
        print(lower_bound, upper_bound)
        in_income_band = (employment_income >= lower_bound) & (employment_income < upper_bound)

        y_subset = earnings[earnings.employment_income_lower_bound == lower_bound][earnings.employment_income_upper_bound == upper_bound]

        income_str = f"{lower_bound}_{upper_bound}"
        matrix[f"employment_income/{income_str}/count"] = sim.map_result(
            in_income_band, "person", "household"
        )

        income_count = y_subset.employment_income_count.values

        y[f"employment_income/{income_str}/count"] = income_count

        matrix[f"employment_income/{income_str}/amount"] = sim.map_result(
            in_income_band * employment_income, "person", "household"
        )

        income_amount = y_subset.employment_income_amount.values

        y[f"employment_income/{income_str}/amount"] = income_amount

    age = sim.calculate("age", period=2025)

    for lower_age in range(0, 80, 10):
        upper_age = lower_age + 10

        in_age_band = (age >= lower_age) & (age < upper_age)

        age_str = f"{lower_age}_{upper_age}"
        matrix[f"age/{age_str}"] = sim.map_result(
            in_age_band, "person", "household"
        )

        age_count = ages[
            [str(age) for age in range(lower_age, upper_age)]
        ].sum(axis=1)

        age_str = f"{lower_age}_{upper_age}"
        y[f"age/{age_str}"] = age_count.values

    return matrix, y


matrix, y = create_target_matrix()

m_national, y_national = create_national_target_matrix(
    "enhanced_frs_2022_23", 2022
)

# Weights - 650 x 100180
original_weights = np.log(sim.calculate("household_weight", 2022).values / 650)
weights = torch.tensor(
    np.ones((650, 100180)) * original_weights,
    dtype=torch.float32,
    requires_grad=True,
)
metrics = torch.tensor(matrix.values, dtype=torch.float32)
weighted_metrics = weights.unsqueeze(-1) * metrics.unsqueeze(0)
totals = weighted_metrics.sum(dim=1)
y = torch.tensor(y.values, dtype=torch.float32)
matrix_national = torch.tensor(m_national.values, dtype=torch.float32)
y_national = torch.tensor(y_national.values, dtype=torch.float32)


def loss(w):
    pred_c = (w.unsqueeze(-1) * metrics.unsqueeze(0)).sum(dim=1)
    mse_c = torch.mean((pred_c / (1 + y) - 1) ** 2)

    pred_n = (w.sum(axis=0) * matrix_national.T).sum(axis=1)
    mse_n = torch.mean((pred_n / (1 + y_national) - 1) ** 2)

    return mse_c + mse_n


optimizer = torch.optim.Adam([weights], lr=0.5)

desc = tqdm(range(1000))

for epoch in desc:
    optimizer.zero_grad()
    l = loss(torch.exp(weights))
    desc.set_description(f"Loss: {l.item()}")
    l.backward()
    optimizer.step()

final_weights = torch.exp(weights).detach().numpy()

with h5py.File("weights.h5", "w") as f:
    f.create_dataset("weights", data=final_weights)

[     0  12570  15000  20000  30000  40000  50000  70000 100000 150000
 200000 300000 500000]
0 12570


ValueError: Length of values (632) does not match length of index (650)

In [55]:
from policyengine_core.reforms import Reform

emp_ni = Reform.from_dict(
    {
        "gov.contrib.policyengine.employer_ni.employee_incidence": {
            "2025-01-01.2025-12-31": 0.4,
            "2026-01-01.2026-12-31": 0.5,
            "2027-01-01.2027-12-31": 0.6,
            "2028-01-01.2028-12-31": 0.7,
            "2028-01-01.2029-12-31": 0.7,
        },
        "gov.hmrc.national_insurance.class_1.rates.employer": {
            "2025-01-01.2100-12-31": 0.15
        },
        "gov.hmrc.national_insurance.class_1.thresholds.secondary_threshold": {
            "2025-01-01.2100-12-31": 96.14
        },
    },
    country_id="uk",
)

baseline = Microsimulation()
reformed = Microsimulation(reform=emp_ni)

gain = reformed.calculate("household_net_income", 2025) - baseline.calculate(
    "household_net_income", 2025
)

In [56]:
weighted_gain = np.dot(final_weights, gain)
populations = np.dot(final_weights, np.ones((100180)))
mean_gain = weighted_gain / populations

In [57]:
incomes

,code,name,total_income_count,total_income_amount
1,E14000554,Berwick-upon-Tweed,37000.0,1.184000e+09
2,E14000569,Bishop Auckland,44000.0,1.315600e+09
3,E14000574,Blaydon,44000.0,1.425600e+09
4,E14000575,Blyth Valley,45000.0,1.305000e+09
5,E14000641,City of Durham,45000.0,1.552500e+09
...,...,...,...,...
657,N06000014,South Antrim,48000.0,1.555200e+09
658,N06000015,South Down,48000.0,1.531200e+09
659,N06000016,Strangford,40000.0,1.272000e+09
660,N06000017,Upper Bann,59000.0,1.728700e+09


In [58]:
df = pd.DataFrame(
    {
        "mean_gain": mean_gain,
        "name": incomes.name.values,
        "code": incomes.code.values,
    }
)

In [59]:
df[df.name == "Oxford East"]

,mean_gain,name,code
448,-192.435414,Oxford East,E14000873


In [60]:
df

,mean_gain,name,code
0,-129.729905,Berwick-upon-Tweed,E14000554
1,-149.162901,Bishop Auckland,E14000569
2,-172.874135,Blaydon,E14000574
3,-148.005294,Blyth Valley,E14000575
4,-127.432858,City of Durham,E14000641
...,...,...,...
645,-176.527914,South Antrim,N06000014
646,-173.620812,South Down,N06000015
647,-141.071970,Strangford,N06000016
648,-165.024939,Upper Bann,N06000017


In [61]:
import plotly.express as px
from policyengine_core.charts import *

fig = px.scatter(df.sort_values("mean_gain"), x="name", y="mean_gain", color_discrete_sequence=[BLUE])
fig.update_layout(
    title="Impact of Autumn Budget employer NI reforms, by Parliamentary constituency"
)
fig.update_layout(
    yaxis_tickformat=",.0f",
    yaxis_tickprefix="£",
    yaxis_title="Average household net income change",
    xaxis_title="Parliamentary constituency",
)

format_fig(fig)

In [62]:
from charts import constituencies

In [63]:
constituencies = constituencies.copy()
constituencies["mean_gain"] = df.set_index("code").loc[constituencies.code].mean_gain.values
constituencies["name"] = df.set_index("code").loc[constituencies.code].name.values

In [64]:
constituencies.x = constituencies.x + constituencies.y % 2 * 0.5

In [67]:
constituencies["size"] = 1

In [102]:
fig = px.scatter(
    constituencies,
    x="x",
    y="y",
    color="mean_gain",
    hover_data=["name"],
    color_continuous_scale=px.colors.diverging.RdBu
).update_layout(
    xaxis_range=[-5, 45],
    yaxis_range=[-5, 45],
    height=580,
    width=700,
)
# Set zero to white
fig.update_coloraxes(cmax=-constituencies.mean_gain.min(), cmin=constituencies.mean_gain.min())

# Set marker to hexagon
fig.update_traces(marker=dict(symbol="hexagon", size=11))

fig.update_layout(
    title="Impact of Autumn Budget employer NI reforms",
    xaxis_tickvals=[],
    xaxis_title="",
    yaxis_tickvals=[],
    yaxis_title="",
    # No bg
    plot_bgcolor="rgba(0,0,0,0)",
    legend_title="Average household net income change",
)